In [2]:
# ==============================================================================
# [스크립트 1] 최종 모델 훈련 (개선5: 시간 기반 검증, 개선7: 밤 데이터 제거 적용)
# ==============================================================================
import pandas as pd
import lightgbm as lgb
import numpy as np
import warnings
import gc
import os
import joblib 

warnings.filterwarnings('ignore')

BASE_PROCESSED_FILE = 'processed_train.feather'

# --- 1. 최종 전처리된 데이터 로딩 ---
print(f"'{BASE_PROCESSED_FILE}' 파일을 불러옵니다...")
df = pd.read_feather(BASE_PROCESSED_FILE)
print("데이터 로딩 완료.")

# --- 2. 데이터 분리 (제거) ---
print("183개 발전소 전체 데이터를 학습용으로 준비합니다...")

# --- 3. 최종 학습 데이터 준비 ---
features = [
    #'appr_temp', #'dew_point', 
    'ceiling', 'cloud_b', 
    'precip_1h', 'pressure',
    'temp_b', 'uv_idx',
    'vis', #'wind_dir_b',  
    'wind_spd_b',#UV때문에 더 좋아짐
    'cloud_a', 'humidity', 'rain', 'snow',
    'wind_spd_a',#'pv_id',                                                
    #'coord1', 'coord2', ,#'temp_max', 'temp_min', #'wind_dir_a', #'real_feel_temp', 'real_feel_temp_shade', 
    # 이미 다 압축반영, 좌표는 location이 다 가지고있고ㅇㅇ
    #15개  11개 제외+ 처음부터 7개제외 원본

   #날씨관련
    'is_precipitating','station_temp_diff',
    'cloud_x_hour','uv_idx_x_hour','humidity_x_hour','temp_x_hour','uv_idx_sq','temp_a_sq','station_wind_spd_diff','station_cloud_diff',
    'is_foggy',#가시거리까지23
   #lag날씨
# (새로운 15분, 30분, 1시간 Lag/Rolling/Std 특성)
    'cloud_a_lag_3','cloud_a_rolling_3_mean','cloud_a_rolling_3_std',
    'cloud_a_lag_6','cloud_a_rolling_6_mean','cloud_a_rolling_6_std',
    'cloud_a_lag_12','cloud_a_rolling_12_mean','cloud_a_rolling_12_std',
    'temp_a_lag_3','temp_a_rolling_3_mean','temp_a_rolling_3_std',
    'temp_a_lag_6','temp_a_rolling_6_mean','temp_a_rolling_6_std',
    'temp_a_lag_12','temp_a_rolling_12_mean','temp_a_rolling_12_std',
    'uv_idx_lag_3','uv_idx_rolling_3_mean','uv_idx_rolling_3_std',
    'uv_idx_lag_6','uv_idx_rolling_6_mean','uv_idx_rolling_6_std',
    'uv_idx_lag_12','uv_idx_rolling_12_mean','uv_idx_rolling_12_std',
    'pressure_lag_3','pressure_rolling_3_mean','pressure_rolling_3_std',
    'pressure_lag_6','pressure_rolling_6_mean','pressure_rolling_6_std',
    'pressure_lag_12','pressure_rolling_12_mean','pressure_rolling_12_std',
    'humidity_lag_3','humidity_rolling_3_mean','humidity_rolling_3_std',
    'humidity_lag_6','humidity_rolling_6_mean','humidity_rolling_6_std',
    'humidity_lag_12','humidity_rolling_12_mean','humidity_rolling_12_std',

    #군집

    'location_cluster','climate_cluster',#2

    #물리기반순환
    'seasonal_hour',
    'sun_exposure_factor', 'diurnal_temp_range',
    'saturation_deficit',
    'hour_sin', 'hour_cos',#'decimal_hour',
    'day_of_year_sin', 'day_of_year_cos',
    'hour', #min,'day_of_year'제외 #11개
    'daylight_cosine',

    #풍향
    'wind_power_a','wind_power_b',#'temp_a_sq','uv_idx_sq',
    #6

    #일몰 일출 특성변수
    'sunset_decimal','sunrise_decimal',
    
    'cloud_anomaly','uv_anomaly','temp_anomaly','humidity_anomaly','vis_anomaly','wind_anomaly','pressure_anomaly',
    'cluster_hour_humidity_mean','cluster_hour_temp_mean','cluster_hour_uv_mean','cluster_hour_cloud_mean','cluster_hour_vis_mean',
    'cluster_hour_wind_mean','cluster_hour_pressure_mean'
]
target = 'nins'

# 183개 전체 데이터를 X_train, y_train으로 정의
X_train = df[features]
y_train = df[target]

# df에서 헬퍼 컬럼을 가져옴
decimal_hours = df['decimal_hour'].to_numpy()
sunrise_times = df['sunrise_decimal'].to_numpy()
sunset_times = df['sunset_decimal'].to_numpy()#이게 테스트파일생성코드에서 맵을 이용해 정의한것? 아 애초에 train파일에 있던 값이네
pv_id_helper = df['pv_id']#목록에서 pvid제거시

#if 'pv_id' in X_train.columns:
#    X_train['pv_id'] = X_train['pv_id'].astype('category')
if 'location_cluster' in X_train.columns:
    X_train['location_cluster'] = X_train['location_cluster'].astype('category')
if 'climate_cluster' in X_train.columns:
    X_train['climate_cluster'] = X_train['climate_cluster'].astype('category')

print("개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 데이터를 '학습에서 제외'합니다 (개선7 적용)...")
gc.collect()

# 기본값 설정
default_sunrise = 4.5 
default_sunset = 20.5 
sunrise_times[sunrise_times < -900] = default_sunrise #NUN값들 즉 병합실패한 값들을 기본값으로 설정해줌
sunset_times[sunset_times < -900] = default_sunset # 군집별 일출, 일몰시간임
sunrise_times[sunrise_times == 0] = default_sunrise#마찬가지 물리적으로 저 시간대가 0일리는 없으니 오류겠지하는거임
sunset_times[sunset_times == 0] = default_sunset

# '밤' 시간 마스크 생성
train_morning_mask = (decimal_hours <= sunrise_times)#일출시간보다 작음 즉 0이겠지
train_evening_mask = (decimal_hours >= sunset_times)#일몰후의 시간 즉 0이겠지
train_night_mask = train_morning_mask | train_evening_mask#아침또는 밤을 만족하는 즉 위의 두값에 포함되는 시간대는 night다 0이다


# [개선7 적용] '밤' 시간 데이터 제거 이미 test에 적용이 됐으니 그리고 위의 과정으로 확실히 했으니 굳이 복잡하게 학습 할 필요는 없어서 제
# (O) 개선 방식: '밤'이 아닌 '낮' 시간 데이터만 선택

train_day_mask = ~train_night_mask # '밤' 마스크의 반대(~) = '낮'

print(f"원본 데이터 {len(X_train)} 행에서 '밤' 시간대 {np.sum(train_night_mask)} 행을 제외합니다.")

# '낮' 시간 데이터만 남김 (X_train, y_train 모두 동일한 마스크 적용)


#-------------------------------------------------------------------------------------------------------------
# ★★★ (개선5) 시간 기반 분리 적용 (이미 '낮' 데이터만 남은 상태에서 수행) 

print("시간 기반으로 '낮' 훈련/검증 세트를 분리합니다 (개선5 적용)...")

X_train = X_train.loc[train_day_mask]
y_train = y_train.loc[train_day_mask]
pv_id_helper = pv_id_helper.loc[train_day_mask]


print(f" -> '낮' 시간 학습 데이터: {len(X_train)} 행")

# 원본 df 메모리 정리
del df, decimal_hours, sunrise_times, sunset_times, train_night_mask, train_day_mask
# (time_helper도 여기서 del 됩니다)
gc.collect()

# [오류 수정] 'pv_id' 기반 분리 (하드코딩 대신 실제 ID 사용)
print("발전소 ID 기반으로 훈련/검증 세트를 분리합니다...")

# 1. '낮' 데이터에 존재하는 *모든* 고유 pv_id 목록을 가져옵니다.

VALIDATION_PV_IDS = [ 
    138, 139, 107, 14, 16, 164, 167, 169, 170, 193,
    22, 28, 32, 45, 51, 56, 59, 77, 80, 86,9
] 

valid_idx_mask = pv_id_helper.isin(VALIDATION_PV_IDS)#수동으로 정한 리스트에 포함됐는지 확인
train_idx_mask = ~valid_idx_mask # (valid의 반대 = train)

print(f"검증용 발전소 ({len(VALIDATION_PV_IDS)}개 수동 지정): {VALIDATION_PV_IDS}")
# 3. 'pv_id' 컬럼을 사용하여 인덱스 마스크 생성

# 4. .loc를 사용하여 분리
X_train_final = X_train.loc[train_idx_mask]# 특정 시간대만 가져오기 위한것 위에서 만들었던거 이용
y_train_final = y_train.loc[train_idx_mask]

X_valid_final = X_train.loc[valid_idx_mask]
y_valid_final = y_train.loc[valid_idx_mask]

# 5. (안전 장치) 분리 후 검증 세트가 비어있는지 다시 확인
if X_valid_final.empty:

    exit
else:
    print(f"   -> 훈련: {len(X_train_final)} 행, 검증: {len(X_valid_final)} 행")

# (메모리 정리)
# (time_helper는 이미 del 되었으므로 리스트에서 제외)
del X_train, y_train, train_idx_mask, valid_idx_mask
gc.collect()

#-------------------------------------------------------------------------------------------------------------
# (이하 final_model.fit(...) 코드는 동일)
# ...
#-------------------------------------------------------------------------------------------------------------

# --- 5. '기본 파라미터'로 모델 생성 ---
final_model = lgb.LGBMRegressor(
    objective='mae',
    metric='mae',
    random_state=42,
    n_jobs=-1,
    n_estimators=3000, 
)

# --- 6. '검증 세트'로 학습 (★수정됨★) ---
print("\n'훈련 데이터'로 학습하고 '검증 데이터'로 조기 종료를 시작합니다...")
final_model.fit(
    X_train_final, y_train_final,
    eval_set=[(X_valid_final, y_valid_final)],
    callbacks=[lgb.early_stopping(350, verbose=True)],
    categorical_feature=['location_cluster','climate_cluster']#pvid제거
)

#-------------------------------------------------------------------------
# 훈련된 모델의 '특성 중요도' 확인

print("\n" + "="*50)
print("모델 훈련 완료. '특성 중요도'를 분석합니다...")

# 1. 특성 이름(features)과 중요도 점수(importances)를 DataFrame으로 합칩니다.

importance_df = pd.DataFrame({
    'feature': features,
    'importance': final_model.feature_importances_
})

# 2. 중요도(importance)가 높은 순서대로 정렬합니다.
importance_df = importance_df.sort_values(by='importance', ascending=False)

print("\n[상위 50개 중요 특성]:")
print(importance_df.head(50))

# 3. 중요도가 0인, 즉 모델이 '전혀 사용하지 않은' 특성을 찾습니다.
zero_importance_features = importance_df[importance_df['importance'] == 0]
print("\n" + "="*50)
print(f"[중요도 0인 특성 (제거 후보): {len(zero_importance_features)}개]")
print(zero_importance_features['feature'].tolist())
print("="*50 + "\n")

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

# --- 7. 최종 모델 저장 ---
print("최종 모델을 'final_model.joblib'로 저장합니다...")
joblib.dump(final_model, 'final_model.joblib5')

print(f"학습에 사용된 {len(features)}개 특성 리스트를 'final_features.joblib'로 저장합니다...")
joblib.dump(features, 'final_features.joblib')

print("저장 완료.")

'processed_train.feather' 파일을 불러옵니다...
데이터 로딩 완료.
183개 발전소 전체 데이터를 학습용으로 준비합니다...
개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 데이터를 '학습에서 제외'합니다 (개선7 적용)...
원본 데이터 19236948 행에서 '밤' 시간대 5888904 행을 제외합니다.
시간 기반으로 '낮' 훈련/검증 세트를 분리합니다 (개선5 적용)...


MemoryError: Unable to allocate 4.67 GiB for an array with shape (94, 13348044) and data type float32

In [5]:
# ==============================================================================
# [스크립트 1] 최종 모델 훈련 (발전소 3개 검증 / 데이터 누수 위험 있음 앞의 발전소군집을 통해 뒤의 발전소 값을 엿보는)

import pandas as pd
import lightgbm as lgb
import numpy as np
import warnings
import gc
import os
import joblib 

warnings.filterwarnings('ignore')

BASE_PROCESSED_FILE = 'processed_train.feather'

# --- 1. 최종 전처리된 데이터 로딩 ---
print(f"'{BASE_PROCESSED_FILE}' 파일을 불러옵니다...")
df = pd.read_feather(BASE_PROCESSED_FILE)
print("데이터 로딩 완료.")

# --- 2. 데이터 분리 (제거) ---
print("183개 발전소 전체 데이터를 학습용으로 준비합니다...")

# --- 3. 최종 학습 데이터 준비 ---
features = [
    # (이전과 동일한 features 리스트...)
    'ceiling', 'cloud_b', 'precip_1h', 'pressure', 'temp_b', 'uv_idx',
    'vis', 'wind_spd_b', 'cloud_a', 'humidity', 'rain', 'snow',
    'wind_spd_a', 'pv_id', 'is_precipitating', 'station_temp_diff',
    'cloud_x_hour', 'uv_idx_x_hour', 'humidity_x_hour', 'temp_x_hour',
    'uv_idx_sq', 'temp_a_sq', 'station_wind_spd_diff', 'station_cloud_diff', 'is_foggy',
    'cloud_a_lag_3', 'cloud_a_rolling_3_mean', 'cloud_a_rolling_3_std',
    'cloud_a_lag_6', 'cloud_a_rolling_6_mean', 'cloud_a_rolling_6_std',
    'cloud_a_lag_12', 'cloud_a_rolling_12_mean', 'cloud_a_rolling_12_std',
    'temp_a_lag_3', 'temp_a_rolling_3_mean', 'temp_a_rolling_3_std',
    'temp_a_lag_6', 'temp_a_rolling_6_mean', 'temp_a_rolling_6_std',
    'temp_a_lag_12', 'temp_a_rolling_12_mean', 'temp_a_rolling_12_std',
    'uv_idx_lag_3', 'uv_idx_rolling_3_mean', 'uv_idx_rolling_3_std',
    'uv_idx_lag_6', 'uv_idx_rolling_6_mean', 'uv_idx_rolling_6_std',
    'uv_idx_lag_12', 'uv_idx_rolling_12_mean', 'uv_idx_rolling_12_std',
    'pressure_lag_3', 'pressure_rolling_3_mean', 'pressure_rolling_3_std',
    'pressure_lag_6', 'pressure_rolling_6_mean', 'pressure_rolling_6_std',
    'pressure_lag_12', 'pressure_rolling_12_mean', 'pressure_rolling_12_std',
    'humidity_lag_3', 'humidity_rolling_3_mean', 'humidity_rolling_3_std',
    'humidity_lag_6', 'humidity_rolling_6_mean', 'humidity_rolling_6_std',
    'humidity_lag_12', 'humidity_rolling_12_mean', 'humidity_rolling_12_std',
    'location_cluster', 'climate_cluster', 'seasonal_hour',
    'sun_exposure_factor', 'diurnal_temp_range', 'saturation_deficit',
    'hour_sin', 'hour_cos', 'day_of_year_sin', 'day_of_year_cos', 'hour',
    'daylight_cosine', 'wind_power_a', 'wind_power_b', 'sunset_decimal',
    'sunrise_decimal', 'cloud_anomaly', 'uv_anomaly', 'temp_anomaly',
    'humidity_anomaly', 'vis_anomaly', 'wind_anomaly', 'pressure_anomaly',
    'cluster_hour_humidity_mean', 'cluster_hour_temp_mean',
    'cluster_hour_uv_mean', 'cluster_hour_cloud_mean',
    'cluster_hour_vis_mean', 'cluster_hour_wind_mean', 'cluster_hour_pressure_mean'
]
target = 'nins'

# (안전장치) df에 없는 특성 자동 제외
features = [f for f in features if f in df.columns]

X_train = df[features]
y_train = df[target]
if 'pv_id' in X_train.columns:
    X_train['pv_id'] = X_train['pv_id'].astype('category')
if 'location_cluster' in X_train.columns:
    X_train['location_cluster'] = X_train['location_cluster'].astype('category')
if 'climate_cluster' in X_train.columns:
    X_train['climate_cluster'] = X_train['climate_cluster'].astype('category')


# ★★★ '밤 데이터 제거' 로직 (먼저 실행) ★★★
print("개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 데이터를 '학습에서 제외'합니다...")
decimal_hours = df['decimal_hour'].to_numpy()
sunrise_times = df['sunrise_decimal'].to_numpy()
sunset_times = df['sunset_decimal'].to_numpy()

default_sunrise = 4.5 
default_sunset = 20.5 
sunrise_times[sunrise_times < -900] = default_sunrise 
sunset_times[sunset_times < -900] = default_sunset 
sunrise_times[sunrise_times == 0] = default_sunrise
sunset_times[sunset_times == 0] = default_sunset

train_morning_mask = (decimal_hours <= sunrise_times)
train_evening_mask = (decimal_hours >= sunset_times)
train_night_mask = train_morning_mask | train_evening_mask
train_day_mask = ~train_night_mask # '낮' 데이터만 True

# '낮' 시간 데이터만 남김 (X_train, y_train, time_helper 모두 동일한 마스크 적용)
X_train = X_train.loc[train_day_mask]
y_train = y_train.loc[train_day_mask]


print(f" -> '낮' 시간 학습 데이터: {len(X_train)} 행")

# 원본 df 메모리 정리
del df, decimal_hours, sunrise_times, sunset_times, train_night_mask, train_day_mask
# (time_helper도 여기서 del 됩니다)
gc.collect()

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# [오류 수정] 'pv_id' 기반 분리 (하드코딩 대신 실제 ID 사용)
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

print("발전소 ID 기반으로 훈련/검증 세트를 분리합니다...")

# 1. '낮' 데이터에 존재하는 *모든* 고유 pv_id 목록을 가져옵니다.
all_pv_ids_in_day_data = X_train['pv_id'].unique()

# 2. 이 목록에서 검증용으로 사용할 3개의 ID를 *자동으로* 선택합니다.
# (예: 목록의 마지막 3개 ID. [0:3]으로 바꾸면 처음 3개 ID)
VALIDATION_PV_IDS = all_pv_ids_in_day_data[-3:] 
print(f"검증용 발전소 (3개): {VALIDATION_PV_IDS}")

# 3. 'pv_id' 컬럼을 사용하여 인덱스 마스크 생성
valid_idx_mask = X_train['pv_id'].isin(VALIDATION_PV_IDS)
train_idx_mask = ~valid_idx_mask # (valid의 반대 = train)

# 4. .loc를 사용하여 분리
X_train_final = X_train.loc[train_idx_mask]
y_train_final = y_train.loc[train_idx_mask]

X_valid_final = X_train.loc[valid_idx_mask]
y_valid_final = y_train.loc[valid_idx_mask]

# 5. (안전 장치) 분리 후 검증 세트가 비어있는지 다시 확인
if X_valid_final.empty:
    print("="*50)
    print("!! 치명적 오류: 검증 세트(X_valid_final)가 비어있습니다.")
    print("   'VALIDATION_PV_IDS'에 문제가 없는지 확인하십시오.")
    print("="*50)
    # (여기서 스크립트를 중지시키는 것이 좋지만, 일단 진행합니다.)
else:
    print(f"   -> 훈련: {len(X_train_final)} 행, 검증: {len(X_valid_final)} 행")

# (메모리 정리)
# (time_helper는 이미 del 되었으므로 리스트에서 제외)
del X_train, y_train, train_idx_mask, valid_idx_mask
gc.collect()

#-------------------------------------------------------------------------------------------------------------
# (이하 final_model.fit(...) 코드는 동일)
# ...
#-------------------------------------------------------------------------------------------------------------

# --- 5. '기본 파라미터'로 모델 생성 ---
final_model = lgb.LGBMRegressor(
    objective='mae',
    metric='mae',
    random_state=42,
    n_jobs=-1,
    n_estimators=3000, 
)

# --- 6. '검증 세트'로 학습 (★수정됨★) ---
print("\n'훈련 데이터'로 학습하고 '검증 데이터'로 조기 종료를 시작합니다...")
final_model.fit(
    X_train_final, y_train_final,
    eval_set=[(X_valid_final, y_valid_final)],
    callbacks=[lgb.early_stopping(350, verbose=True)],
    categorical_feature=['pv_id','location_cluster','climate_cluster']
)

#-------------------------------------------------------------------------
# 훈련된 모델의 '특성 중요도' 확인

print("\n" + "="*50)
print("모델 훈련 완료. '특성 중요도'를 분석합니다...")

# 1. 특성 이름(features)과 중요도 점수(importances)를 DataFrame으로 합칩니다.

importance_df = pd.DataFrame({
    'feature': features,
    'importance': final_model.feature_importances_
})

# 2. 중요도(importance)가 높은 순서대로 정렬합니다.
importance_df = importance_df.sort_values(by='importance', ascending=False)

print("\n[상위 50개 중요 특성]:")
print(importance_df.head(50))

# 3. 중요도가 0인, 즉 모델이 '전혀 사용하지 않은' 특성을 찾습니다.
zero_importance_features = importance_df[importance_df['importance'] == 0]
print("\n" + "="*50)
print(f"[중요도 0인 특성 (제거 후보): {len(zero_importance_features)}개]")
print(zero_importance_features['feature'].tolist())
print("="*50 + "\n")

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

# --- 7. 최종 모델 저장 ---
print("최종 모델을 'final_model.joblib'로 저장합니다...")
joblib.dump(final_model, 'final_model.joblib5')

print(f"학습에 사용된 {len(features)}개 특성 리스트를 'final_features.joblib'로 저장합니다...")
joblib.dump(features, 'final_features.joblib')

print("저장 완료.")

'processed_train.feather' 파일을 불러옵니다...
데이터 로딩 완료.
183개 발전소 전체 데이터를 학습용으로 준비합니다...
개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 데이터를 '학습에서 제외'합니다...
 -> '낮' 시간 학습 데이터: 13348044 행
발전소 ID 기반으로 훈련/검증 세트를 분리합니다...
검증용 발전소 (3개): ['PV_ID_97', 'PV_ID_98', 'PV_ID_99']
Categories (183, object): ['PV_ID_0', 'PV_ID_1', 'PV_ID_100', 'PV_ID_101', ..., 'PV_ID_96', 'PV_ID_97', 'PV_ID_98', 'PV_ID_99']
   -> 훈련: 13135614 행, 검증: 212430 행

'훈련 데이터'로 학습하고 '검증 데이터'로 조기 종료를 시작합니다...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.996915 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23365
[LightGBM] [Info] Number of data points in the train set: 13135614, number of used features: 100
[LightGBM] [Info] Start training from score 129.100006
Training until validation scores don't improve for 350 rounds
Early stopping, best iteration is:
[224]	valid_0's l1: 72.3818

모델 훈련 완료. '특성 

In [6]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import warnings
import gc
import os
import joblib 

# --- 1. 파일 경로 설정 ---
# (모델과 특성 파일은 '모델저장파일코드'에서 생성된 파일과 이름이 같아야 합니다)
MODEL_FILE = 'final_model.joblib5'          # ★ 사용하시는 모델 파일명
FEATURES_FILE = 'final_features.joblib'   # ★ '모델저장파일코드'에서 저장한 특성 리스트 파일
PROCESSED_TEST_FILE = 'processed_test.feather'   # 전처리된 테스트 파일
SUBMISSION_TEMPLATE = 'submission_sample.csv'
SUBMISSION_FILE = 'submission.csv'

# --- 2. 모델 및 '특성 리스트' 로드 ---
print(f"훈련된 최종 모델 ({MODEL_FILE})과 특성 리스트 ({FEATURES_FILE})를 불러옵니다...")
final_model = joblib.load(MODEL_FILE)
features = joblib.load(FEATURES_FILE)
            
print(f"-> 모델 로드 완료. 예측에 사용할 {len(features)}개 특성 리스트 로드 완료.")

# ★★★ (핵심 수정 2) 하드코딩된 'features = [...]' 리스트 전체 삭제 ★★★
# (이제 'features' 변수는 파일에서 자동으로 로드되므로 하드코딩된 리스트는 필요 없습니다)

# --- 3. '공통 데이터'(test.csv) 로딩 및 전처리 ---
print(f"'{PROCESSED_TEST_FILE}' 파일을 불러옵니다...")
# '동적 마스크'에 헬퍼(Helper) 컬럼이 필요하므로 전체를 다 불러옵니다
test_df_processed = pd.read_feather(PROCESSED_TEST_FILE)

# ★★★ (핵심 수정 3) X_test는 파일에서 로드된 'features' 변수를 사용 ★★★
X_test = test_df_processed[features]

# (훈련(fit) 때와 동일하게 두 컬럼 모두 category로 변환)
# (만약 'pv_id'나 'location_cluster'가 features 리스트에 없다면 이 부분은 자동으로 무시됩니다)
if 'pv_id' in features:
    X_test['pv_id'] = X_test['pv_id'].astype('category') 
if 'location_cluster' in features:
    X_test['location_cluster'] = X_test['location_cluster'].astype('category') 
if 'climate_cluster' in features:
    X_test['climate_cluster'] = X_test['climate_cluster'].astype('category')

# --- 4. 최종 예측 수행 ---
print("최종 예측을 수행합니다...")
predictions = final_model.predict(X_test)

# --- 5. (필수) 후처리 2단계 적용 ---
# 5A. (기본) 0보다 작은 값은 0으로 보정
predictions[predictions < 0] = 0 

# 5B. (핵심) '동적 0 처리' (안전망)
print("개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 예측을 0으로 강제 조정합니다...")
# 'processed_test.feather'에서 헬퍼(Helper) 컬럼을 가져옴
decimal_hours = test_df_processed['decimal_hour'].to_numpy()
sunrise_times = test_df_processed['sunrise_decimal'].to_numpy()
sunset_times = test_df_processed['sunset_decimal'].to_numpy()

# (훈련 코드와 동일한 로직) 혹시모를 안전장치인거지
default_sunrise = 4.5
default_sunset = 20.5
sunrise_times[sunrise_times < -900] = default_sunrise 
sunset_times[sunset_times < -900] = default_sunset
sunrise_times[sunrise_times == 0] = default_sunrise
sunset_times[sunset_times == 0] = default_sunset

pred_morning_mask = (decimal_hours <= sunrise_times)
pred_evening_mask = (decimal_hours >= sunset_times)
pred_night_mask = pred_morning_mask | pred_evening_mask

# 예측(predictions) 배열에 0 처리 적용
predictions[pred_night_mask] = 0
print("동적 마스크 적용 완료.")


# --- 6. 제출 파일(submission.csv) 생성 ---
print("제출 파일을 생성합니다...")
submission_df = pd.read_csv(SUBMISSION_TEMPLATE) 
submission_df['nins'] = predictions
submission_df.to_csv(SUBMISSION_FILE, index=False)

print("="*50)
print(f"최종 제출 파일 '{SUBMISSION_FILE}' 생성이 완료되었습니다.")

훈련된 최종 모델 (final_model.joblib5)과 특성 리스트 (final_features.joblib)를 불러옵니다...
-> 모델 로드 완료. 예측에 사용할 100개 특성 리스트 로드 완료.
'processed_test.feather' 파일을 불러옵니다...
최종 예측을 수행합니다...
개인화된 '일출/일몰 맵'을 기반으로 밤 시간대 예측을 0으로 강제 조정합니다...
동적 마스크 적용 완료.
제출 파일을 생성합니다...
최종 제출 파일 'submission.csv' 생성이 완료되었습니다.
